In [1]:
digests = {
    "trypsin": {
        "start" : [
            {
                "amino_acid": "D", 
                "cut_position": "left"
            }
        ], 
        "end" : [
            {
                "amino_acid": "R", 
                "cut_position": "right"
            }, {
                "amino_acid": "K", 
                "cut_position": "right"
            }
        ]
    }
}

In [6]:
from collections import namedtuple

Database = namedtuple(
    'Database', 
    ['fasta_file', 'proteins', 'kmers', 'b_hits', 'y_hits'], 
    defaults=['', {}, {}, [], []]
)

prot_entry = namedtuple(
    'prot_entry', 
    ['sequence']
)

In [50]:
def digest(db: Database, digest_type: str, missed_cleavages: int) -> Database:
    '''
    Digest each protein in the database. If no digest is done, then 
    the original database is returned. 
    NOTE: 
    The entires in the database after digestion are the names of the form
    <protein_name>_<start_position>_<end_position>

    Inputs:
        db:                 (Database) the input source
        digest_type:        (str) the digestion to perform
        missed_cleavages:   (int) the number of missed cleavages allowed
    Outputs:
        (Database) updated protein entries
    '''
    if digest_type not in digests:
        return db
    
    digest_rules = digests[digest_type]
    starts = {s['amino_acid']: s['cut_position'] for s in digest_rules['start']}
    ends = {s['amino_acid']: s['cut_position'] for s in digest_rules['end']}
    
    new_prots = {}
    
    for p_name, entry in db.proteins.items():
        
        # keep track of what gets digested
        digested = []
        
        for pos, aa in enumerate(entry.sequence):
            
            if aa in starts:
                
                # get the starting position for this cut based on rule
                s = pos if starts[aa] == 'left' else pos + 1
                
                allowed_misses = missed_cleavages
                
                # find all of the next ends. we will keep track of them for up to missed_cleavages
                for j in range(pos, len(entry.sequence)):
                    
                    # if we're out of missed cleavages, break
                    if allowed_misses < 0:
                        break
                    
                    # check if we're at the end
                    if j == len(entry.sequence) - 1:
                        
                        # get the cut sequence
                        digested.append((entry.sequence[s:], s, len(entry.sequence)))
                        break
                    
                    # check of this aa is an end
                    if entry.sequence[j] in ends:
                        
                        # first reduce allowed
                        allowed_misses -= 1
                        
                        # determine if we do j or j+1 based on the rule
                        e = j if ends[entry.sequence[j]] == 'left' else j + 1
                        
                        digested.append((entry.sequence[s:e], s, e))
                        
        for d in digested:
            new_prots[f'{p_name}_{d[1]}_{d[2]}'] = prot_entry(d[0])
            
    db = db._replace(proteins=new_prots)
    return db

In [51]:
p = 'MATPEASGSGEKVEGSEPSVTYYRLEEVAKRNSAEETWMVIHGRVYDITRFLSEHPGGEEVLLEQAGADATESFEDVGHSPDAREMLKQYYIGDVHPSDLKPKGDDKDPSKNNSCQSSWAYWFVPIVGAILIGFLYRHFWADSKSS'
db = Database('', {'prot_1': prot_entry(p)})

In [52]:
print([i for i, aa in enumerate(p) if aa == 'K' or aa == 'R'])

[11, 23, 29, 30, 43, 49, 83, 87, 100, 102, 106, 110, 136, 143]


In [53]:
db2 = digest(db, 'trypsin', 1)

new prots: 
{'prot_1_46_50': prot_entry(sequence='DITR'), 'prot_1_46_84': prot_entry(sequence='DITRFLSEHPGGEEVLLEQAGADATESFEDVGHSPDAR'), 'prot_1_68_84': prot_entry(sequence='DATESFEDVGHSPDAR'), 'prot_1_68_88': prot_entry(sequence='DATESFEDVGHSPDAREMLK'), 'prot_1_75_84': prot_entry(sequence='DVGHSPDAR'), 'prot_1_75_88': prot_entry(sequence='DVGHSPDAREMLK'), 'prot_1_81_84': prot_entry(sequence='DAR'), 'prot_1_81_88': prot_entry(sequence='DAREMLK'), 'prot_1_93_101': prot_entry(sequence='DVHPSDLK'), 'prot_1_93_103': prot_entry(sequence='DVHPSDLKPK'), 'prot_1_98_101': prot_entry(sequence='DLK'), 'prot_1_98_103': prot_entry(sequence='DLKPK'), 'prot_1_104_107': prot_entry(sequence='DDK'), 'prot_1_104_111': prot_entry(sequence='DDKDPSK'), 'prot_1_105_107': prot_entry(sequence='DK'), 'prot_1_105_111': prot_entry(sequence='DKDPSK'), 'prot_1_107_111': prot_entry(sequence='DPSK'), 'prot_1_107_137': prot_entry(sequence='DPSKNNSCQSSWAYWFVPIVGAILIGFLYR'), 'prot_1_141_144': prot_entry(sequence='DSK'),

In [54]:
db2.proteins

{'prot_1_46_50': prot_entry(sequence='DITR'),
 'prot_1_46_84': prot_entry(sequence='DITRFLSEHPGGEEVLLEQAGADATESFEDVGHSPDAR'),
 'prot_1_68_84': prot_entry(sequence='DATESFEDVGHSPDAR'),
 'prot_1_68_88': prot_entry(sequence='DATESFEDVGHSPDAREMLK'),
 'prot_1_75_84': prot_entry(sequence='DVGHSPDAR'),
 'prot_1_75_88': prot_entry(sequence='DVGHSPDAREMLK'),
 'prot_1_81_84': prot_entry(sequence='DAR'),
 'prot_1_81_88': prot_entry(sequence='DAREMLK'),
 'prot_1_93_101': prot_entry(sequence='DVHPSDLK'),
 'prot_1_93_103': prot_entry(sequence='DVHPSDLKPK'),
 'prot_1_98_101': prot_entry(sequence='DLK'),
 'prot_1_98_103': prot_entry(sequence='DLKPK'),
 'prot_1_104_107': prot_entry(sequence='DDK'),
 'prot_1_104_111': prot_entry(sequence='DDKDPSK'),
 'prot_1_105_107': prot_entry(sequence='DK'),
 'prot_1_105_111': prot_entry(sequence='DKDPSK'),
 'prot_1_107_111': prot_entry(sequence='DPSK'),
 'prot_1_107_137': prot_entry(sequence='DPSKNNSCQSSWAYWFVPIVGAILIGFLYR'),
 'prot_1_141_144': prot_entry(sequence='

In [55]:
db.proteins

{'prot_1': prot_entry(sequence='MATPEASGSGEKVEGSEPSVTYYRLEEVAKRNSAEETWMVIHGRVYDITRFLSEHPGGEEVLLEQAGADATESFEDVGHSPDAREMLKQYYIGDVHPSDLKPKGDDKDPSKNNSCQSSWAYWFVPIVGAILIGFLYRHFWADSKSS')}